In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
import numpy as np


In [2]:
# 학습셋의 변형을 설정하는 부분

train_datagen = ImageDataGenerator(rescale=1./255,# 주어진 이미지의 크기를 설정
                                   horizontal_flip=True, # 수평 대칭 이미지를 50% 확률로 만듬
                                   width_shift_range=0.1, # 전체 크기의 15% 범위에서 좌우로 이동
                                   height_shift_range=0.1, # 위, 아래로 이동
                                   rotation_range=5, # 정해진 각도만큼 회전
                                   shear_range=0.7, # 좌표 하나를 고정시키고 나머지를 이동
                                   zoom_range=1.2, # 확대 또는 축소
                                   vertical_flip=True, # 수직 대칭 이미지를 만듦
                                   fill_mode='nearest' # 빈 공간을 채우는 방법, nearest : 비슷한 색으로                              
                                  )   

train_generator = train_datagen.flow_from_directory('./data/train', # 학습 셋이 있는 폴더 위치
                                                    target_size = (150,150), # 사이즈 조절
                                                    batch_size = 5,
                                                    class_mode = 'binary'
                                                   )

Found 160 images belonging to 2 classes.


In [3]:
# 테스트 셋 설정
# 이미지를 부풀리지 않아도 됨.
test_datagen = ImageDataGenerator(rescale=1./255)   

test_generator = train_datagen.flow_from_directory('./data/test', # 학습 셋이 있는 폴더 위치
                                                    target_size = (150,150), # 사이즈 조절
                                                    batch_size = 5,
                                                    class_mode = 'binary'
                                                   )

Found 120 images belonging to 2 classes.


In [5]:
# CNN 모델 만들어서 적용
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(learning_rate=0.0002),metrics=['accuracy'])

espc = EarlyStopping(monitor='val_loss',patience=5)

his = model.fit(train_generator,epochs=100,validation_data=test_generator,validation_steps=10,callbacks=[espc])

Epoch 1/100


32/32 [==============================] - 5s 104ms/step - loss: 0.7753 - accuracy: 0.4750 - val_loss: 0.6927 - val_accuracy: 0.4800
Epoch 2/100
32/32 [==============================] - 3s 79ms/step - loss: 0.7049 - accuracy: 0.4938 - val_loss: 0.6982 - val_accuracy: 0.4400
Epoch 3/100
32/32 [==============================] - 2s 76ms/step - loss: 0.6882 - accuracy: 0.5562 - val_loss: 0.6897 - val_accuracy: 0.5200
Epoch 4/100
32/32 [==============================] - 2s 74ms/step - loss: 0.7017 - accuracy: 0.4812 - val_loss: 0.6907 - val_accuracy: 0.4800
Epoch 5/100
32/32 [==============================] - 2s 73ms/step - loss: 0.6946 - accuracy: 0.4750 - val_loss: 0.6872 - val_accuracy: 0.6400
Epoch 6/100
32/32 [==============================] - 2s 75ms/step - loss: 0.6908 - accuracy: 0.5125 - val_loss: 0.6876 - val_accuracy: 0.6000
Epoch 7/100
32/32 [==============================] - 2s 74ms/step - loss: 0.6881 - accuracy: 0.5250 - val_loss: 0.6965 - val_accuracy: 0.4000
Epo

In [12]:
from tensorflow.keras import Input,models,layers,metrics,optimizers
from tensorflow.keras.applications import VGG16

In [14]:
# 전이 학습
train_datagen = ImageDataGenerator(rescale = 1./255,# 주어진 이미지의 크기를 설정
                                   horizontal_flip = True, # 수평 대칭 이미지를 50% 확률로 만듬
                                   width_shift_range = 0.1, # 전체 크기의 15% 범위에서 좌우로 이동
                                   height_shift_range = 0.1, # 위, 아래로 이동
                                   rotation_range = 5, # 정해진 각도만큼 회전
                                   shear_range = 0.7, # 좌표 하나를 고정시키고 나머지를 이동
                                   zoom_range = 1.2, # 확대 또는 축소
                                   vertical_flip = True, # 수직 대칭 이미지를 만듦
                                   fill_mode = 'nearest' # 빈 공간을 채우는 방법, nearest : 비슷한 색으로                              
                                  )   

train_generator = train_datagen.flow_from_directory('./data/train', # 학습 셋이 있는 폴더 위치
                                                    target_size = (150,150), # 사이즈 조절
                                                    batch_size = 5,
                                                    class_mode = 'binary'
                                                   )

test_datagen = ImageDataGenerator(rescale=1./255)   

test_generator = train_datagen.flow_from_directory('./data/test', # 학습 셋이 있는 폴더 위치
                                                    target_size = (150,150), # 사이즈 조절
                                                    batch_size = 5,
                                                    class_mode = 'binary'
                                                   )

# VGG16 모델
transfer_model = VGG16(weights='imagenet',include_top=False,input_shape=(150,150,3))
transfer_model.trainable = False
transfer_model.summary()

Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
58889256/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 12

In [15]:
# 나의 모델 설정
finetune_model = models.Sequential()
finetune_model.add(transfer_model)

finetune_model.add(Flatten())

finetune_model.add(Dense(64))
finetune_model.add(Activation('relu'))
finetune_model.add(Dropout(0.5))
finetune_model.add(Activation('relu'))

finetune_model.add(Dense(1))
finetune_model.add(Activation('sigmoid'))
finetune_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_3 (Dense)             (None, 64)                524352    
                                                                 
 activation_7 (Activation)   (None, 64)                0         
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 activation_8 (Activation)   (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                

In [16]:
finetune_model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(learning_rate=0.0002),metrics=['accuracy'])

espc = EarlyStopping(monitor='val_loss',patience=5)

his = finetune_model.fit(train_generator,epochs=100,validation_data=test_generator,validation_steps=10,callbacks=[espc])

Epoch 1/100
32/32 [==============================] - 9s 254ms/step - loss: 0.7156 - accuracy: 0.5188 - val_loss: 0.6522 - val_accuracy: 0.7000
Epoch 2/100
32/32 [==============================] - 8s 244ms/step - loss: 0.7096 - accuracy: 0.5312 - val_loss: 0.7673 - val_accuracy: 0.4200
Epoch 3/100
32/32 [==============================] - 8s 246ms/step - loss: 0.7100 - accuracy: 0.5500 - val_loss: 0.6404 - val_accuracy: 0.6600
Epoch 4/100
32/32 [==============================] - 8s 244ms/step - loss: 0.6415 - accuracy: 0.6125 - val_loss: 0.5929 - val_accuracy: 0.7200
Epoch 5/100
32/32 [==============================] - 8s 244ms/step - loss: 0.7155 - accuracy: 0.5312 - val_loss: 0.6437 - val_accuracy: 0.7200
Epoch 6/100
32/32 [==============================] - 8s 246ms/step - loss: 0.6533 - accuracy: 0.6125 - val_loss: 0.6392 - val_accuracy: 0.6800
Epoch 7/100
32/32 [==============================] - 8s 240ms/step - loss: 0.6425 - accuracy: 0.6000 - val_loss: 0.6350 - val_accuracy: 0.6200